**MODEL V8**

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
import supervision as sv
#NEED TO DOWNLOAD AND UPLOAD VIDEO IF YOU USING GG COLAB
# Define the video path
MARKET_SQUARE_VIDEO_PATH = "/content/market-square.mp4"

# Initialize the YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the video file
cap = cv2.VideoCapture(MARKET_SQUARE_VIDEO_PATH)

# Verify the output directory and permissions
output_dir = "/content"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.access(output_dir, os.W_OK):
    raise PermissionError(f"Write permission denied for the directory {output_dir}")

# Define the output video path
output_path = os.path.join(output_dir, "object_counting_output_v8_3.mp4")

assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize VideoWriter with a successful FourCC code
fourcc_code = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_path, fourcc_code, fps, (w, h))

if not video_writer.isOpened():
    raise IOError(f"Error initializing video writer with path {output_path}")

# Custom function to filter only human detections
def filter_human_detections(detections):
    human_class_id = 0
    mask = detections.class_id == human_class_id
    filtered_detections = sv.Detections(
        xyxy=detections.xyxy[mask],
        confidence=detections.confidence[mask],
        class_id=detections.class_id[mask]
    )
    return filtered_detections

# Process video frames
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object detection
    results = model(frame, imgsz=1280)

    # Extract bounding boxes, confidences, and class IDs
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

    # Create detections object
    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=class_ids
    )

    # Filter only human detections
    human_detections = filter_human_detections(detections)

    # Annotate the frame
    annotated_frame = box_annotator.annotate(scene=frame, detections=human_detections)

    # Write the processed frame to the output video
    video_writer.write(annotated_frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved to {output_path}")
